In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import os
import sys
import datetime
import glob as glob
import numpy as np
import cv2

import keras
import tensorflow as tf
from keras.applications.inception_v3 import InceptionV3, preprocess_input
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D,Dropout
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from keras.optimizers import SGD
from tensorflow.keras.models import load_model
import pandas as pd
import matplotlib.pyplot as plt
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing import image

import warnings
warnings.filterwarnings("ignore")




In [ ]:
def preprocess(img):
    width, height = img.shape[0], img.shape[1]
    img = image.array_to_img(img, scale=False)

    # Crop 48x48px
    desired_width, desired_height = 224, 224

    if width < desired_width:
        desired_width = width
    start_x = np.maximum(0, int((width-desired_width)/2))

    img = img.crop((start_x, np.maximum(0, height-desired_height), start_x+desired_width, height))
    img = img.resize((224, 224))

    img = image.img_to_array(img)
    return img / 255.

In [ ]:
def preprocess_data(train_dir, validate_dir):
    img_width, img_height = 224,224
    batch_size =16
    # data pre-processing for training
    train_datagen =  ImageDataGenerator(
        rescale=1./255,
        rotation_range = 10,
        zoom_range = [1.00, 1.00],
        horizontal_flip = True,
        fill_mode = 'nearest')
    

    # data pre-processing for validation
    validate_datagen =  ImageDataGenerator(rescale=1./255)

    # generate and store training data
    train_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size = (img_width, img_height),
        batch_size = batch_size)

    # generate and store validation data
    validate_generator = validate_datagen.flow_from_directory(
        validate_dir,
        target_size = (img_width, img_height),
        batch_size = batch_size)
    
    return train_generator, validate_generator

In [ ]:
train_dir = "../input/chest-xray-pneumonia/chest_xray/train"
validation_dir = "../input/chest-xray-pneumonia/chest_xray/val"
test_dir = "../input/chest-xray-pneumonia/chest_xray/test"
# train_generator, validate_generator = preprocess_data(train_dir, validate_dir)

In [ ]:
# Prepare data augmentation configuration
train_datagen = ImageDataGenerator(rotation_range=40,
                                   rescale=1./255,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True,
                                   validation_split=0.1)

test_datagen = ImageDataGenerator(rescale=1./255)


train_generator = train_datagen.flow_from_directory(train_dir,
                                                    target_size=(224,224),
                                                    batch_size=16,
                                                    class_mode='categorical')

validation_generator = train_datagen.flow_from_directory(validation_dir,
                                                    target_size=(224,224),
                                                    batch_size=16,
                                                    class_mode='categorical'
                                                        )

test_generator = test_datagen.flow_from_directory(test_dir,
                                                  target_size=(224,224),
                                                  batch_size=16,
                                                  class_mode='categorical'
                                                 )

In [ ]:
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D



In [ ]:
nb_classes=2

incepV3_model = InceptionV3(weights = 'imagenet', include_top = False, 
                            input_shape =(224, 224,3))
for layer in incepV3_model.layers:
    layer.trainable = False

x = incepV3_model.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.3)(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(nb_classes, activation = 'softmax')(x)
model = Model(incepV3_model.input, predictions)
model.compile(optimizer=SGD(lr=1e-4, momentum=0.9), loss='categorical_crossentropy')
model.summary()


In [ ]:
NUM_EPOCHS = 30
batch_size=8
history = model.fit_generator(
    train_generator,
        epochs = NUM_EPOCHS,
        validation_data=validation_generator,
    callbacks=None)

In [ ]:
BATCH_SIZE=16
history = model.fit_generator(
    train_generator,
    steps_per_epoch=len(train_generator.filenames) / BATCH_SIZE,
    epochs=10
    validation_data=validate_generator,
    validation_steps=len(validate_generator.filenames) / BATCH_SIZE,
    callbacks=None)